주의: 깃헙 리포 페이제이 링크된 mybinder를 통해 웹브라우저에서 클라우드를 통해 실행하는 도커 이미지에는 BNFC-meta는 포함되어 있지 않고 아래 `install.sh`도 성공하지 못할 것이므로 이 노트북은 실행되지 않음! 추가 라이브러리 설치가 가능할 정도로 리소스가 충분한 도커 이미지 실행 환경에서만 가능함.

In [1]:
:!./install.sh -- 성공적으로 설치되면 상단바의 리로드 ⟳ 버튼 누르기

    BNFC-meta-0.6.1

In [2]:
{-# LANGUAGE QuasiQuotes #-}
{-# LANGUAGE TemplateHaskell #-}

module Expr where

import Language.LBNF

bnfc [lbnf|

    EAdd. Expr   ::= Expr  "+" Expr1 ; -- expr
    EMul. Expr1  ::= Expr1 "*" Expr2 ; -- term
    EInt. Expr2  ::= Integer ;       -- factor

 -- coercions
    _.  Expr  ::= Expr1 ;  -- a term can be an expr
    _.  Expr1 ::= Expr2 ;  -- a factor can be a term
--    _.  Expr2 ::= "(" Expr ")" ; -- any parenthesized expr is a term

    entrypoints Expr;
|]

In [3]:
import Expr

:info Expr
:type expr

expr :: QuasiQuoter

data Expr = EAdd Expr Expr | EMul Expr Expr | EInt Integer 	-- Defined at Expr.hs:8:1
instance Show Expr -- Defined at Expr.hs:8:1
instance Ord Expr -- Defined at Expr.hs:8:1
instance Eq Expr -- Defined at Expr.hs:8:1

In [4]:
:type [expr| 2 + 3 * 4 |]

[expr| 2 + 3 * 4 |] :: Expr

In [5]:
[expr| 2 + 3 * 4 |]

EAdd (EInt 2) (EMul (EInt 3) (EInt 4))

In [6]:
:type pExpr
:type tokens

pExpr :: [Token] -> ParseMonad Expr

tokens :: String -> [Token]

In [7]:
tokens "2 + 3 * 4"
pExpr (tokens "2 + 3 * 4 ")

[PT (Pn 0 1 1) (TI "2"),PT (Pn 2 1 3) (TS "+" 2),PT (Pn 4 1 5) (TI "3"),PT (Pn 6 1 7) (TS "*" 1),PT (Pn 8 1 9) (TI "4")]

Ok (EAdd (EInt 2) (EMul (EInt 3) (EInt 4)))

In [8]:
calc :: Expr -> Integer
calc (EAdd e1 e2) = calc e1 + calc e2
calc (EMul e1 e2) = calc e1 * calc e2
calc (EInt v)     = v

In [9]:
calc [expr| 2 + 3 * 4 |]

14

In [10]:
calc <$> pExpr (tokens " 2 + 3 * 4 ")

Ok 14

In [11]:
interpreter = (calc <$>) . pExpr . tokens

:type interpreter

interpreter :: String -> ParseMonad Integer

In [12]:
interpreter " 2 + 3 * 4 "

Ok 14

In [13]:
calc [expr| (2 + 3) * 4 |]

: 

In [14]:
interpreter " (2 + 3) * 4 "

: 

----

위에 있는 BNFC 명세는 Happy 파서 명세를 자동으로 만들어내고 그로부터 Happy 파서 생성기가 코드를 생성한다.
(Haskell이 아닌 다른 언어로 뽑아내는 옵션을 주면 그 언어용 파서 생성기 명세를 자동으로 만들어내 활용)

BNFC의 한단계 아래에 있는 Happy 명세는 대략 이런 모양이다.
참고로 C/C++의 Yacc이나 Bison 파서 명세도 대략 비슷하게 생겼는데 왜냐하면,
많은 LR기반의 파서 생성기들이 Yacc을 본따서 만들었기 때문.

````
{
module Expr where

data Expr = EAdd Expr Expr
          | EMul Expr Expr
          | EInt Integer
}

... 뭔가 여기 오는 것들이 조금 있는데 생략

%token

... 토큰 정의도 생략 ...

%%

Expr :: {Expr}
  : Expr '+' Expr1      { EAdd $1 $3 }
  | Expr1               { $1 }

Expr1 :: {Expr}
  : Expr1 '*' Expr2     { EMul $1 $3 }
  | Expr2               { $1 }

Expr2 :: {Expr}
  : int                 { EInt $1 }
  | '(' Expr ')'        { $2 }
  
{}
````

----

일반적으로
 * LR계열 parsing
    - 장점:
       * left-recursive 문법 명세가 자유로움
    - 단점: 
       * 내부동작을 문법분석 이론에 대해 공부해본 사람이 아니면 개념을 잡기가 어려움
       * 에러 메시지 내용이나 가리키는 위치가 비직관적인 경우가 발생할 소지가 많음
       
 * LL계열 parsing
    - 단점:
       * left-recursive 문법 명세가 귀찮음  
    - 장점:
       * 내부동작을 재귀함수 구현으로 이해하면 직관적
       * 에러 메시지 위치나 내용이 비교적 직관적

LR 계열의 bottom-up / table-driven 파서 생성기만 활용하냐 하면 그건 또 아님.

LL 계열의 top-down / recursive-descendent 파서 생성기 및 라이브러리도 나름 많이 활용됨.

 * ANTLR같이 Java 등에서 많이 활용되는 (다른 언어로도 파서를 생성) 파서 생성기가 있다.<br>
    - 이건 Happy보다는 BNFC의 수준에 있다고 보면 된다. (참고로 BNFC도 여러 언어로 파서를 생성)<br>
    - left-recursive grammar 처리의 귀찮음을 다른 편의 기능으로 (예를 들면 문법분석된 나무구조를 확인해볼 수 있는 GUI를 함께 제공하는 등) 커버하고 있다고 볼 수 있음.
 * 소스코드 생성기가 아닌 라이브러리 형태 주로 Parser Combinator 라이브러리라라 부르는 것들도 있다.<br>
    - 대표적인 것이 Haskell의 Parsec이 있고 이것을 본따서 (혹은 비슷한 아이디어로 조금 다르게) 다른 여러 언어에도


파서 소스코드 생성기 vs. 파서 라이브러리
 * 소스코드 생성기
    - 장점: 최적화된 소스코드 생성 가능
    - 단점: 파서 명세와 함께 파서를 사용하는 SW 빌드에 프로그래밍언어 컴파일러 외에 별도의 도구에 의존성 증가
 * 파서 라이브러리
    - 장점: SW 빌드에 프로그래밍언어 컴파일러 외에 별도의 도구에 의존성 증가 (파서 라이브러리도 해당 언어로 작성된 라이브러리일 뿐) 
    - 단점: LR같은 table driven 방식이라면 동적인 데이타 구조로 처리하는 것이 소스코드 생성에 비해 성능이 점 떨어질 수도 있음


LL 그리고 LR 관련 기초 이론이 정리된지 수십년이지만 아직도 파서 구현에 대한 연구는 계속되고 있다.
참고로 최근 (2020년 01월) 정보과학회논문지에도 http://168.131.151.113/paper/kiise202001.pdf 이런 논문이 발표되고 있다.
이 수업을 들었으면 이런 논문을 봤을 때 자세한 내용까지는 혹시 이해가 안가더라도 어떤 걸 하고 있구나 하는 감을 잡을 수 있으면 이번 학기 수업의 나름의 목적을 달성했다고 생각합니다. 

----

참고자료

* http://168.131.151.113/paper/kiise202001.pdf
* http://ANTLR.org
* https://github.com/westes/flex
* https://www.gnu.org/software/bison/
* https://happy
* https://bnfc.digitalgrammars.com/
* https://github.com/haskell/parsec
* https://en.wikipedia.org/wiki/Comparison_of_parser_generators